In [2]:
# Cell 1: Imports and Setup
import json
import random
from pathlib import Path
from pprint import pprint
import pandas as pd

# Dataset paths
DATASET_DIR = Path("resource/dataset")
SA_SGG_PATH = DATASET_DIR / "generation" / "sa-sgg.jsonl"
MA_SGG_PATH = DATASET_DIR / "generation" / "ma-sgg.jsonl"
SGQA_PATH = DATASET_DIR / "understanding" / "sgqa.jsonl"
SGDS_PATH = DATASET_DIR / "understanding" / "sgds.jsonl"

print("Dataset paths configured.")

Dataset paths configured.


In [3]:
# Cell 2: Helper Functions

def load_jsonl(path):
    """Load a JSONL file into a list of dictionaries."""
    with open(path, 'r') as f:
        return [json.loads(line) for line in f]

def sample_data(data, n=1, seed=None):
    """Randomly sample n items from data."""
    if seed is not None:
        random.seed(seed)
    return random.sample(data, min(n, len(data)))

def display_sample(sample, title="Sample"):
    """Pretty print a sample with a title."""
    print(f"\n{'='*80}")
    print(f"{title}")
    print('='*80)
    pprint(sample, width=100, depth=4)

print("Helper functions loaded.")

Helper functions loaded.


In [4]:
# Load SA-SGG dataset
sa_sgg_data = load_jsonl(SA_SGG_PATH)

print(f"SA-SGG Dataset Statistics:")
print(f"  Total samples: {len(sa_sgg_data)}")
print(f"  Fields: {list(sa_sgg_data[0].keys())}")

# Sample and display
sample = sample_data(sa_sgg_data, n=1)[0]

print("\n" + "="*80)
print("RANDOM SA-SGG SAMPLE")
print("="*80)

print(f"\n[Target Sentence]")
print(f"  {sample['target_sentence']}")

print(f"\n[Context]")
print(f"  {sample['context'][:300]}..." if len(sample.get('context', '')) > 300 else f"  {sample.get('context', '(none)')}")

print(f"\n[Ground Truth Scene Graph(s)]")
for i, graph in enumerate(sample['graphs']):
    print(f"  Graph {i+1} (action_id: {graph['action_id']}):")
    for triplet in graph['triplets']:
        print(f"    {triplet[0]} -> {triplet[1]} -> {triplet[2]}")

print(f"\n[Mandatory Space (allowed vocabulary)]")
print(f"  Verbs: {sample['mandatory_space']['verb'][:8]}...")
print(f"  Objects: {sample['mandatory_space']['object'][:8]}...")
print(f"  Relationships: {sample['mandatory_space']['relationship']}")

SA-SGG Dataset Statistics:
  Total samples: 1188
  Fields: ['data_id', 'doc_index', 'text_part_index', 'context', 'target_sentence', 'graph_id', 'is_repeatable', 'graphs', 'verbs', 'mandatory_space', 'context_graphs']

RANDOM SA-SGG SAMPLE

[Target Sentence]
  the bolt was loosened using the wrench.

[Context]
  The engine cover was securely shut with both hands, ensuring everything was properly sealed and in place. The individual approached the basin, ready to freshen up. The tap was turned on, allowing liquid to flow for a thorough hand wash. Once the hands were clean, the tap was turned off to conserve l...

[Ground Truth Scene Graph(s)]
  Graph 1 (action_id: 3):
    person -> verb -> loosen
    loosen -> dobj -> bolt
    loosen -> with -> wrench
    loosen -> with -> hand1

[Mandatory Space (allowed vocabulary)]
  Verbs: ['close', 'hold', 'insert', 'loosen', 'move', 'open', 'pick-up', 'place']...
  Objects: ['person', 'bolt', 'engine-cover', 'hand1', 'hand2', 'part', 'sink', 'tap']

In [5]:
# Load MA-SGG dataset
ma_sgg_data = load_jsonl(MA_SGG_PATH)

print(f"MA-SGG Dataset Statistics:")
print(f"  Total samples: {len(ma_sgg_data)}")
print(f"  Fields: {list(ma_sgg_data[0].keys())}")

# Find a sample with multiple graphs
multi_graph_samples = [s for s in ma_sgg_data if len(s['graphs']) > 1]
sample = sample_data(multi_graph_samples, n=1)[0]

print("\n" + "="*80)
print("RANDOM MA-SGG SAMPLE (with multiple actions)")
print("="*80)

print(f"\n[Target Sentence]")
print(f"  {sample['target_sentence']}")

print(f"\n[Number of Actions]: {len(sample['graphs'])}")

print(f"\n[Ground Truth Scene Graphs]")
for i, graph in enumerate(sample['graphs']):
    print(f"\n  Scene Graph {i+1} (action_id: {graph['action_id']}):")
    for triplet in graph['triplets']:
        print(f"    {triplet[0]} -> {triplet[1]} -> {triplet[2]}")

MA-SGG Dataset Statistics:
  Total samples: 853
  Fields: ['data_id', 'doc_index', 'text_part_index', 'context', 'target_sentence', 'graph_id', 'is_repeatable', 'graphs', 'verbs', 'mandatory_space', 'context_graphs']

RANDOM MA-SGG SAMPLE (with multiple actions)

[Target Sentence]
  The garment was positioned on the panel with both hands, ensuring it was ready for the next step.

[Number of Actions]: 3

[Ground Truth Scene Graphs]

  Scene Graph 1 (action_id: 1):
    pick-up -> dobj -> shirt
    pick-up -> from -> table
    pick-up -> with -> hand1
    pick-up -> with -> hand2
    person -> verb -> pick-up

  Scene Graph 2 (action_id: 2):
    carry -> dobj -> shirt
    carry -> to -> board
    carry -> with -> hand1
    carry -> with -> hand2
    person -> verb -> carry

  Scene Graph 3 (action_id: 3):
    place -> dobj -> shirt
    place -> on -> board
    place -> with -> hand1
    place -> with -> hand2
    person -> verb -> place


In [6]:
# Load SGQA dataset
sgqa_data = load_jsonl(SGQA_PATH)

total_qa_pairs = sum(len(s['qa_pairs']) for s in sgqa_data)

print(f"SGQA Dataset Statistics:")
print(f"  Total samples: {len(sgqa_data)}")
print(f"  Total QA pairs: {total_qa_pairs}")
print(f"  Avg QA pairs per sample: {total_qa_pairs / len(sgqa_data):.1f}")
print(f"  Fields: {list(sgqa_data[0].keys())}")

# Sample and display
sample = sample_data(sgqa_data, n=1)[0]

print("\n" + "="*80)
print("RANDOM SGQA SAMPLE")
print("="*80)

print(f"\n[Context Graphs] ({len(sample['context_graphs'])} scene graphs)")
for i, graph in enumerate(sample['context_graphs'][:3]):  # Show first 3
    print(f"\n  Graph {i+1}:")
    for triplet in graph[:4]:  # Show first 4 triplets
        print(f"    {triplet[0]} -> {triplet[1]} -> {triplet[2]}")
    if len(graph) > 4:
        print(f"    ... ({len(graph) - 4} more triplets)")
if len(sample['context_graphs']) > 3:
    print(f"\n  ... ({len(sample['context_graphs']) - 3} more graphs)")

print(f"\n[Question-Answer Pairs] ({len(sample['qa_pairs'])} pairs)")
for i, qa in enumerate(sample['qa_pairs']):
    print(f"\n  Q{i+1}: {qa['Q']}")
    print(f"  A{i+1}: {qa['A']}")

SGQA Dataset Statistics:
  Total samples: 100
  Total QA pairs: 500
  Avg QA pairs per sample: 5.0
  Fields: ['data_id', 'doc_index', 'text_part_index', 'context_graphs', 'qa_pairs']

RANDOM SGQA SAMPLE

[Context Graphs] (29 scene graphs)

  Graph 1:
    pick-up -> dobj -> cardboard
    cardboard -> from -> floor
    pick-up -> with -> hand1
    person -> verb -> pick-up

  Graph 2:
    place -> dobj -> cardboard
    place -> on -> workbench
    place -> with -> hand1
    person -> verb -> place

  Graph 3:
    pick-up -> dobj -> wood
    pick-up -> with -> hand1
    pick-up -> with -> hand2
    person -> verb -> pick-up

  ... (26 more graphs)

[Question-Answer Pairs] (5 pairs)

  Q1: What object was picked up immediately before opening the paint-can?
  A1: screwdriver

  Q2: Which tool was used last before the first paint application began?
  A2: paintbrush

  Q3: What object was manipulated three times consecutively with the same tool?
  A3: paint

  Q4: What was the first object pl

In [7]:
# Load SGDS dataset
sgds_data = load_jsonl(SGDS_PATH)

print(f"SGDS Dataset Statistics:")
print(f"  Total samples: {len(sgds_data)}")
print(f"  Fields: {list(sgds_data[0].keys())}")

# Sample and display
sample = sample_data(sgds_data, n=1)[0]

print("\n" + "="*80)
print("RANDOM SGDS SAMPLE")
print("="*80)

print(f"\n[Scene Graph Triplets]")
for triplet in sample['triplet']:
    print(f"  {triplet[0]} -> {triplet[1]} -> {triplet[2]}")

print(f"\n[Candidate Descriptions]")
for i, desc in enumerate(sample['variations']):
    marker = " <-- CORRECT" if i == sample['position'] else ""
    print(f"  [{chr(65+i)}] {desc}{marker}")

print(f"\n[Correct Answer]: {chr(65 + sample['position'])} (position {sample['position']})")

SGDS Dataset Statistics:
  Total samples: 250
  Fields: ['target_sentence', 'position', 'variations', 'triplet', 'context_graphs']

RANDOM SGDS SAMPLE

[Scene Graph Triplets]
  person -> verb -> tighten
  tighten -> dobj -> screw
  tighten -> on -> socket
  tighten -> with -> screwdriver
  screwdriver -> in -> hand1

[Candidate Descriptions]
  [A] The fastener on the receptacle was then secured securely using the tool. <-- CORRECT
  [B] The fastener on the receptacle was then loosened gently using the tool.
  [C] The fastener on the receptacle was then removed entirely without any tools.
  [D] The fastener on the receptacle was then painted over using a brush.
  [E] The fastener on the receptacle was then ignored completely despite having the tool.

[Correct Answer]: A (position 0)


In [8]:
# Summary table
total_qa = sum(len(s['qa_pairs']) for s in sgqa_data)

summary = pd.DataFrame({
    'Task': ['SA-SGG', 'MA-SGG', 'SGQA', 'SGDS'],
    'Type': ['Generation', 'Generation', 'Understanding', 'Understanding'],
    'Samples': [len(sa_sgg_data), len(ma_sgg_data), len(sgqa_data), len(sgds_data)],
    'Description': [
        'Single-Action Scene Graph Generation',
        'Multi-Action Scene Graph Generation', 
        f'Scene Graph QA ({total_qa} QA pairs)',
        'Scene Graph Description Selection'
    ]
})

print("TSG-Bench Dataset Summary")
print("="*80)
print(summary.to_string(index=False))
print(f"\nTotal samples: {summary['Samples'].sum()}")

TSG-Bench Dataset Summary
  Task          Type  Samples                          Description
SA-SGG    Generation     1188 Single-Action Scene Graph Generation
MA-SGG    Generation      853  Multi-Action Scene Graph Generation
  SGQA Understanding      100        Scene Graph QA (500 QA pairs)
  SGDS Understanding      250    Scene Graph Description Selection

Total samples: 2391
